# Explore Dataset - Homework exercice 2 (Programming task)

## Introduction

In this work we will analyse the dataset named [MAGIC Gamma Telescope](https://archive.ics.uci.edu/ml/datasets/MAGIC+Gamma+Telescope). 
The data are MC generated to simulate registration of high energy gamma particles in a ground-based atmospheric Cherenkov gamma telescope using the imaging technique. Cherenkov gamma telescope observes high energy gamma rays, taking advantage of the radiation emitted by charged particles produced inside the electromagnetic showers initiated by the gammas, and developing in the atmosphere. 

The main goal is to apply features selection methods to properly prepare the dataset to train a model.

One of the greater challenge in machine learning is selecting the best features to train the model. We need only the features which are highly dependent on the output variable.



### Authors:

- Catarina Silva


In [ ]:
#Import Pandas library
import pandas as pd

#Import Numpy library
import numpy as np

#Import Plotly library
import plotly.express as px

#Import Matplotlib library
import matplotlib.pyplot as plt

#Import Seaborn library
from pylab import rcParams
import seaborn as sb

#Import Scipy's Pearson Correllation function
import scipy
from scipy.stats.stats import pearsonr

#Import SKlearn Scaler, PCA & KPCA
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA, KernelPCA

In [ ]:
#Define plot libraries settings
#Python magic function that allows plot inside Jupyter Notebook
%matplotlib inline 
rcParams['figure.figsize'] = 12, 8
sb.set_style('whitegrid')

In [ ]:
#Center notebook images
from IPython.core.display import HTML
HTML("""
<style>
.output_png {
    display: table-cell;
    text-align: center;
    vertical-align: middle;
}
</style>
""")

## Load dataset

In [ ]:
file = './dataset/magic04.data'

dataset = pd.read_csv(file)

dataset.columns = ['fLength', 'fWidth', 'fSize', 'fConc', 'fConc1', 'fAsym', 'fM3Long', 'f3MTrans', 'fAlpha', 'fDist', 'class']
dataset

In [ ]:
dataset['class'].value_counts()

In [ ]:
sb.pairplot(dataset, hue='class', diag_kind="kde")

In [ ]:
sb.pairplot(dataset, hue='class', diag_kind="kde", kind = 'reg')

## Apply Pearson’s correlation to detect linear correlations

In [ ]:
corr = dataset.corr(method='pearson').abs()
corr

In [ ]:
sb.heatmap(corr, xticklabels=corr.columns.values, yticklabels=corr.columns.values, annot=True, cmap=plt.cm.Reds)

In [ ]:
#Threshold for cutoff
th = 0.75

# Select upper triangle of correlation matrix
upper = corr.where(np.triu(np.ones(corr.shape), k=1).astype(np.bool))

# Find index of feature columns with correlation greater than threshold
to_drop = [column for column in upper.columns if any(upper[column] > th)]
print(to_drop)

In [ ]:
corr = dataset.drop(dataset[to_drop], axis=1).corr(method='pearson').abs()

In [ ]:
sb.heatmap(corr, xticklabels=corr.columns.values, yticklabels=corr.columns.values, annot=True, cmap=plt.cm.Reds)

Pearson Correlation is a statistic that measures linear correlation between two features. It has a value between +1 and −1. A value of +1 is total positive linear correlation, 0 is no linear correlation, and −1 is total negative linear correlation.

This correlation can be used to identify pairs of linear correlated features.
Since the features are linear correlated (increase/decrease at similar rates) having both in the dataset does not offer additional information.

This method can be used to removed higly correlated features, however it does not provide any information regarding the relation between the the input feature and the target output.
As such, it is not possible to identify the most 2 discriminative features with this method.

## ANOVA f-test feature ranking

ANOVA (Analysis of Variance) helps us to select the best features.
ANOVA is a parametric statistical hypothesis test for determining whether the means from two or more samples of data (often three or more) come from the same distribution or not.

An F-statistic, or F-test, is a class of statistical tests that calculate the ratio between variances values, such as the variance from two different samples or the explained and unexplained variance by a statistical test, like ANOVA. The ANOVA method is a type of F-statistic referred to here as an ANOVA f-test.

Importantly, it is used when one variable is numeric and one is categorical, such as numerical input variables and a classification target variable in a classification task.

The results of this test can be used for feature selection where those features that are independent of the target variable can be removed from the dataset.

In [ ]:
# Load and summarize the dataset
from pandas import read_csv
from sklearn.model_selection import train_test_split

# Funtion that loads the dataset
def load_dataset(filename):
    # load the dataset as a pandas DataFrame
    data = pd.read_csv(filename, header=None)
    # retrieve numpy array
    dataset = data.values
    # split into input (X) and output (y) variables
    X = dataset[:, :-1]
    y = dataset[:,-1]
    return X, y

# Load the dataset
X, y = load_dataset('./dataset/magic04.data')

# Split into train and test sets (split 70/30)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1)

# Summarize the dataset
print('Train', X_train.shape, y_train.shape)
print('Test', X_test.shape, y_test.shape)

In [ ]:
# ANOVA f-test feature selection for numerical data
from pandas import read_csv
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif
from matplotlib import pyplot
import heapq

# Function that applies feature selection
def select_features(X_train, y_train, X_test):
    # configure to select all features
    fs = SelectKBest(score_func=f_classif, k='all')
    # learn relationship from training data
    fs.fit(X_train, y_train)
    # transform train input data
    X_train_fs = fs.transform(X_train)
    # transform test input data
    X_test_fs = fs.transform(X_test)
    return X_train_fs, X_test_fs, fs

# Feature selection
X_train_fs, X_test_fs, fs = select_features(X_train, y_train, X_test)

# The rank of each feature
for i in range(len(fs.scores_)):
    print('Feature %d: %f' % (i, fs.scores_[i]))

# plot the ranks
pyplot.bar([i for i in range(len(fs.scores_))], fs.scores_)
pyplot.show()

# Define the cutoff for best features
k=2

# Print K best features
idx = heapq.nlargest(k, range(len(fs.scores_)), fs.scores_.__getitem__)
print("The {} top features are:".format(k))
for i in idx:
    print("Feature {}".format(i))

The goal of feature selection is to identify a reduced set of features that are discrimative (are provide enough information to properly separate the target output) but reduce the computational time or even improve accuracy.
This method ranks the relation of each feature with the target output (a higher rank implies that the feature is higly related with the target output).
From the rank of the ANOVA f-test the two most disrciminative features are 8 and 0.

However it is not possible to state that using only these two features are sufficiently to train the model. The third most discriminative feature (number 1) is almost similar (in rank) to feature 0.

## Dimension Reduction

In machine learning, the performance of a model only benefits from more features up until a certain point.
The more features are fed into a model, the more the dimensionality of the data increases.
As the dimensionality increases, overfitting becomes more likely.

Not only can high dimensionality lead to long training times, more features often lead to an algorithm overfitting as it tries to create a model that explains all the features in the data.

There are multiple techniques that can be used to fight overfitting, but dimensionality reduction is one of the most effective techniques.

### PCA

In [ ]:
#Numerical Features
features = dataset.columns
features = features.delete(len(features)-1)

# Separating out the features
x = dataset.loc[:, features].values

# Separating out the target
y = dataset.loc[:,['class']].values

# Color of the output
color=[]
for out in y:
    if out == 0:
        color.append('#1f77b4')
    else:
        color.append('#bcbd22')

# Standardizing the features
x = StandardScaler().fit_transform(x)

In [ ]:
#PCA nº columns
pca = PCA(n_components=2)

#Get Best Features throw PCA
principalComponents = pca.fit_transform(x)

#Resolve info into DataFrame
principalDf = pd.DataFrame(data = principalComponents
             , columns = ['Best Feature 1', 'Best Feature 2'])

In [ ]:
#Append Categorical Feature
finalDf = pd.concat([principalDf, dataset[['class']]], axis = 1)

In [ ]:
# Show DataFrame
print(finalDf)

plt.figure()
plt.scatter(principalComponents[:,0], principalComponents[:,1], c=color)
plt.show()

In the plot it is possible to see the dataset reduced to only two features.
Each axis represent one of those features and the dots are the values of each feature and the color of the dot represents the class.

It is possible to see that there is a large area of overlapping between the blue and green dots, as such the reconstruction using only two feature (with PCA) is not sufficient discriminative.

### KPCA

In [ ]:
#KPCA nº columns
kpca = KernelPCA(n_components=2, kernel='linear')

#Get Best Features throw KPCA
transformer  = kpca.fit_transform(x)

#Resolve info into DataFrame
kpcaDf = pd.DataFrame(data = transformer
             , columns = ['Best Feature 1', 'Best Feature 2'])

In [ ]:
#Append Categorical Feature
kpcaDf = pd.concat([kpcaDf, dataset[['class']]], axis = 1)

In [ ]:
#Show DataFrame
print(kpcaDf)

plt.figure()
plt.scatter(transformer[:,0], transformer[:,1], c=color)
plt.show()

In the plot it is possible to see the dataset reduced to only two features.
Each axis represent one of those features and the dots are the values of each feature and the color of the dot represents the class.



## KPCA (RBF kernel)

In [ ]:
#KPCA nº columns
rbf_kpca = KernelPCA(n_components=2, kernel='rbf')

#Get Best Features throw KPCA
transformer  = rbf_kpca.fit_transform(x)

#Resolve info into DataFrame
rbf_kpcaDf = pd.DataFrame(data = transformer
             , columns = ['Best Feature 1', 'Best Feature 2'])

In [ ]:
#Append Categorical Feature
rbf_kpcaDf = pd.concat([rbf_kpcaDf, dataset[['class']]], axis = 1)

In [ ]:
#Show DataFrame
print(rbf_kpcaDf)

plt.figure()
plt.scatter(transformer[:,0], transformer[:,1], c=color)
plt.show()

The above plot is similar to the ones depicted previously.

Contrary to PCA and KPCA(linear) this shows less overlappings area between the blue and green dots.
As such, these two features provide more information than the reconstruction of PCA KPCA(linear).